In [41]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import csv
import re

In [2]:
#분유 제조사 목록
milkName = ['갈리아', '거버', '남양유업', '네슬레', '네이쳐스원', '노발락', '뉴트리시아', '뢰벤짠', '매일유업', '메이지', '밀라산', '밀루파', 
            '벨라미스오가닉', '아스펜', '아이배냇', '아이엠뉴질랜드', '애보트', '얼스베스트', '엔파밀', '오브맘', '와코도', '일동후디스', '카브리타',
            '커클랜드', '퇴퍼', '파스퇴르', '페디아슈어', '퓨어랜드', '후마나', 'HOLLE', 'Hipp', 'LG생활건강']

In [3]:
slowfood = ['남양유업',  '네슬레', '래퍼티스가든', '맘스쌀과자', '매일유업', '미즈앤코', '베베쿡', '비치넛', '에바토', '일동후디스', '파스퇴르', 
            'LG생활건강', 'SafrunatS.L', '루솔', '뽀뽀뜨', '아이배냇', '얼스베스트', '엘빈즈', '오가닉맘', '짱죽', '팜투베이비', '푸드케어', '풀무원']

In [12]:
gijugi = [ '겐키', '군', '깨끗한나라', '나비잠', '네띠', '네이쳐러브메레', '네추럴블라썸', '네츄라오가닉', '대디베이비', '드라이퍼스', '로맘스', 
          '로한', '리베로', '리보쓰', '리에또', '마르마이플러스', '마미포코', '맘스네이처', '메리즈', '메리트산업', '메이커키즈', '모나리자', 
          '모모래빗', '모이몰른', '몰텍스', '무니', '무루', '무미', '밍크뮤', '바바랩', '밤보', '밤부베베', '백양산업', '범킨스', '베베린도',
           '베베몬', '베베원', '베이비리', '베이비스토리', '베이비앙', '베이비앤아이', '베이비오가닉', '베피스', '벤스랜드', '보솜이', 
          '블루독베이비', '서림', '세븐스제너레이션', '세한', '쇼콜라', '수안산업', '슈퍼대디', '신성', '쌍용C&B', '아가똥', '아가짱', '아이두레',
           '아이씨프로젝트', '아이에이커머스', '아이엠트루', '아이웰', '아이티씨', '암웨이', '압소바', '앙블랑', '애플크럼비', '에뜨와', 
          '에이메르네이쳐', '에코튜드네이처', '에프랑', '엠케이', '영림B&A', '오가닉맘', '오로라', '오보소', '오아이비', '올러브', '유트러스', 
          '은홍섬유', '이마트', '이엔컴퍼니', '이오스트', '일동아이엠씨', '조이로이', '주치니', '주타노다이퍼', '참사랑', '천싸요', '첨이첨이', 
          '치쿨', '캉가케어', '커클랜드', '케이맘', '코트니', '코튼리퍼블릭', '쿠쉬스', '쿠잉', '큐비앤맘', '클로즈', '키즈디조이', '킨도', '킷앤킨',
           '토디앙', '파라솔', '파코라반베이비', '팸퍼스', '페넬로페', '펠트친구', '포그니', '포유', '폴라리스', '폴맘', '피노', '핑고', '하기스', 
          '하이가', '한빛산업', '행키베베', '헤인셀레스티얼', '현대BABY산업', '홈플러스', 'BB DISTRIBE', 'GBSTYLE', 'GERBER', 'MK']

In [14]:
babyCar = [ '구름과환경', '구비', '굿베이비', '그라코', '그린탐', '깜', '나단스', '뉴나', '다이치', '델타', '도렐', '동방레포츠', '라스깔라', '레카로', '로드스타', '리안', 
           '리첼', '리틀타익스', '린덴', '마루스', '마리꼬베베', '마마루', '마마스앤파파스', '마이크라라이트', '맥시코시', '맥클라렌', '몽트뢰', '무무브', '무치', '미마', 
           '바론', '밤비노', '베이비몬스터', '베이비스타일', '베이비시티', '베이비싱', '베이비젠', '베이비조거', '베이비캠프', '베이비트랜드', '베이비프라임', '베이비홈', 
           '보브코리아', '부가부', '브라이택스', '브레비', '뻬그뻬레고', '샤오미', '세기산업', '세이프티퍼스트', '세피앙', '스퀴즈', '스토케', '스트롤에어', '실버크로스', 
           '실버팍스', '싸이벡스', '써니러브', '아가방앤컴퍼니', '아넥스', '아발론', '아이빌리브', '아이사랑', '아이엠베이비', '아이캔디', '아이쿠', '아프리카', '알로앤루', 
           '알퐁소', '앙팡스', '어파베이비', '에그', '에르고베이비', '에스디스피드', '에어보스', '에이블트레이딩', '엔픽스', '엘레갈로', '엘레니어', '엠버', '예떼', '오르빗베이비', 
           '오타브', '와이업', '와이케이비앤씨', '요야', '유니원', '이바겐', '이븐플로', '이지워커', '인스텝', '잉글라차','잉글레시나', '제로', '제인', '조이', '조이파트너스', 
           '조코', '줄즈', '중모토이플러스', '쥬비', '지비', '쭈쭈베베', '차일드홈', '치코', '카펠라', '코뮌코리아', '코사토', '코엔코', '콜크래프트', '콤비', '콩코드', '퀴니', 
           '큐터스', '키도포타머스', '키디', '키즈엠브레이스', '킨더웨건', '킨즈', '테크넘', '토이파파', '톨스토이', '톰비', '툴레', '파파앤코', '팔리', '페도라', '포브', '프시케', 
           '필앤테즈', '하이브리드', '한스', '한일레인보우', '해님', '해밀턴', '해피랜드F&C', '햇님토이', '헨티', 'ABC', 'GB', 'MEIBAO', 'SEEC', 'VOVO', 'belecoo']

In [42]:
def csv2list(filename):
    lists = []
    file = open(filename, "r", encoding = 'utf-8')
    while True:
        line = file.readline().rstrip("\n")
        if line:
            line = line.split(",")
            lists.append(line)
        else:
            break
    return lists

In [7]:
for i in milkName:
    content001 = csv2list(f"../3. data/1. 분유/{i}/nv_{i}_content.csv")
    content002 = csv2list(f"../3. data/1. 분유/{i}/nv_{i}_total.csv")
    content003 = csv2list(f"../3. data/1. 분유/{i}/nv_{i}_feedback.csv")

    content000 = []
    content000.extend(content001)
    content000.extend(content002)
    content000.extend(content003)

    content00 = sum(content000, [])
    token001=re.sub(r'[-=+#/\:^$@*\"※~\&▶%ㆍ!』}{\;\‘|\_(\)\[\]\<\>`\'…》]+',"",str(content00))
    token002=re.sub(r'[0-9A-Za-z]+',"",token001)
    token003=re.sub(r'\\',"",token002)
    token004=re.sub('  ',"",token003)
    token005 = token004.split(',')
    token006 = pd.DataFrame(token005)
    token007 = token006[token006[0].map(len)>2]
    token008 = token007.reset_index(drop=True)

    token008.to_csv(f"./1. 분유/{i}_merge.csv", mode="a", index=False, header=False)

In [8]:
for i in slowfood:
    content001 = csv2list(f"../3. data/2. 이유식/{i}/nv_{i}_content.csv")
    content002 = csv2list(f"../3. data/2. 이유식/{i}/nv_{i}_total.csv")
    content003 = csv2list(f"../3. data/2. 이유식/{i}/nv_{i}_feedback.csv")

    content000 = []
    content000.extend(content001)
    content000.extend(content002)
    content000.extend(content003)

    content00 = sum(content000, [])
    token001=re.sub(r'[-=+#/\:^$@*\"※~\&▶%ㆍ!』}{\;\‘|\_(\)\[\]\<\>`\'…》]+',"",str(content00))
    token002=re.sub(r'[0-9A-Za-z]+',"",token001)
    token003=re.sub(r'\\',"",token002)
    token004=re.sub('  ',"",token003)
    token005 = token004.split(',')
    token006 = pd.DataFrame(token005)
    token007 = token006[token006[0].map(len)>2]
    token008 = token007.reset_index(drop=True)

    token008.to_csv(f"./2. 이유식/{i}_merge.csv", mode="a", index=False, header=False)

In [13]:
for i in gijugi:
    content001 = csv2list(f"../3. data/3. 기저귀/nv_{i}_content.csv")
    content002 = csv2list(f"../3. data/3. 기저귀/nv_{i}_total.csv")
    content003 = csv2list(f"../3. data/3. 기저귀/nv_{i}_feedback.csv")

    content000 = []
    content000.extend(content001)
    content000.extend(content002)
    content000.extend(content003)

    content00 = sum(content000, [])
    token001=re.sub(r'[-=+#/\:^$@*\"※~\&▶%ㆍ!』}{\;\‘|\_(\)\[\]\<\>`\'…》]+',"",str(content00))
    token002=re.sub(r'[0-9A-Za-z]+',"",token001)
    token003=re.sub(r'\\',"",token002)
    token004=re.sub('  ',"",token003)
    token005 = token004.split(',')
    token006 = pd.DataFrame(token005)
    token007 = token006[token006[0].map(len)>2]
    token008 = token007.reset_index(drop=True)

    token008.to_csv(f"./3. 기저귀/{i}_merge.csv", mode="a", index=False, header=False)
#펨퍼스 인코딩하기

겐키
군
깨끗한나라
나비잠
네띠
네이쳐러브메레
네추럴블라썸
네츄라오가닉
대디베이비
드라이퍼스
로맘스
로한
리베로
리보쓰
리에또
마르마이플러스
마미포코
맘스네이처
메리즈
메리트산업
메이커키즈
모나리자
모모래빗
모이몰른
몰텍스
무니
무루
무미
밍크뮤
바바랩
밤보
밤부베베
백양산업
범킨스
베베린도
베베몬
베베원
베이비리
베이비스토리
베이비앙
베이비앤아이
베이비오가닉
베피스
벤스랜드
보솜이
블루독베이비
서림
세븐스제너레이션
세한
쇼콜라
수안산업
슈퍼대디
신성
쌍용C&B
아가똥
아가짱
아이두레
아이씨프로젝트
아이에이커머스
아이엠트루
아이웰
아이티씨
암웨이
압소바
앙블랑
애플크럼비
에뜨와
에이메르네이쳐
에코튜드네이처
에프랑
엠케이
영림B&A
오가닉맘
오로라
오보소
오아이비
올러브
유트러스
은홍섬유
이마트
이엔컴퍼니
이오스트
일동아이엠씨
조이로이
주치니
주타노다이퍼
참사랑
천싸요
첨이첨이
치쿨
캉가케어
커클랜드
케이맘
코트니
코튼리퍼블릭
쿠쉬스
쿠잉
큐비앤맘
클로즈
키즈디조이
킨도
킷앤킨
토디앙
파라솔
파코라반베이비
페넬로페
펠트친구
포그니
포유
폴라리스
폴맘
피노
핑고
하기스
하이가
한빛산업
행키베베
헤인셀레스티얼
현대BABY산업
홈플러스
BB DISTRIBE
GBSTYLE
GERBER
MK


In [15]:
for i in babyCar:
    content001 = csv2list(f"../3. data/4. 유모차/nv_{i}_content.csv")
    content002 = csv2list(f"../3. data/4. 유모차/nv_{i}_total.csv")
    content003 = csv2list(f"../3. data/4. 유모차/nv_{i}_feedback.csv")

    content000 = []
    content000.extend(content001)
    content000.extend(content002)
    content000.extend(content003)

    content00 = sum(content000, [])
    token001=re.sub(r'[-=+#/\:^$@*\"※~\&▶%ㆍ!』}{\;\‘|\_(\)\[\]\<\>`\'…》]+',"",str(content00))
    token002=re.sub(r'[0-9A-Za-z]+',"",token001)
    token003=re.sub(r'\\',"",token002)
    token004=re.sub('  ',"",token003)
    token005 = token004.split(',')
    token006 = pd.DataFrame(token005)
    token007 = token006[token006[0].map(len)>2]
    token008 = token007.reset_index(drop=True)

    token008.to_csv(f"./4. 유모차/{i}_merge.csv", mode="a", index=False, header=False)

구비
굿베이비
그라코
그린탐
깜
나단스
뉴나
다이치
델타
도렐
동방레포츠
라스깔라
레카로
로드스타
리안
리첼
리틀타익스
린덴
마루스
마리꼬베베
마마루
마마스앤파파스
마이크라라이트
맥시코시
맥클라렌
몽트뢰
무무브
무치
미마
바론
밤비노
베이비몬스터
베이비스타일
베이비시티
베이비싱
베이비젠
베이비조거
베이비캠프
베이비트랜드
베이비프라임
베이비홈
보브코리아
부가부
브라이택스
브레비
뻬그뻬레고
샤오미
세기산업
세이프티퍼스트
세피앙
스퀴즈
스토케
스트롤에어
실버크로스
실버팍스
싸이벡스
써니러브
아가방앤컴퍼니
아넥스
아발론
아이빌리브
아이사랑
아이엠베이비
아이캔디
아이쿠
아프리카
알로앤루
알퐁소
앙팡스
어파베이비
에그
에르고베이비
에스디스피드
에어보스
에이블트레이딩
엔픽스
엘레갈로
엘레니어
엠버
예떼
오르빗베이비
오타브
와이업
와이케이비앤씨
요야
유니원
이바겐
이븐플로
이지워커
인스텝
잉글라차
잉글레시나
제로
제인
조이
조이파트너스
조코
줄즈
중모토이플러스
쥬비
지비
쭈쭈베베
차일드홈
치코
카펠라
코뮌코리아
코사토
코엔코
콜크래프트
콤비
콩코드
퀴니
큐터스
키도포타머스
키디
키즈엠브레이스
킨더웨건
킨즈
테크넘
토이파파
톨스토이
톰비
툴레
파파앤코
팔리
페도라
포브
프시케
필앤테즈
하이브리드
한스
한일레인보우
해님
해밀턴
해피랜드F&C
햇님토이
헨티
ABC
GB
MEIBAO
SEEC
VOVO
belecoo
